# Quantum Computing Workshop: Introduction to Qiskit 2

In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter
from qiskit.result import marginal_counts
from qiskit.visualization import *
from qiskit_aer import AerSimulator, Aer
from qiskit.quantum_info import Operator, Statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
with open('api_key.txt', 'r') as file:
    token = file.read()
service = QiskitRuntimeService(channel="ibm_quantum",token=token)

In [ ]:
import numpy as np
from IPython.display import Image
# Ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
aer_backend = AerSimulator()
ibmq_backend = service.get_backend('ibm_brisbane')

## Qiskit IBM Runtime Primitives and Sessions

## Parameterized Rotation Gates

Rotation gates perform a counter clockwise rotation around the X-, Y-, Z- Axis of the Bloch sphere by a given angle. Let's take the Ry gate as an example:

In [ ]:
# Create a parameterized (theta) RY circuit (qc)

# Your code goes here

In [ ]:
visualize_transition(qc.assign_parameters({theta: 2*np.pi}))

We evaluate the measurement probability for |0> and |1> for a range of parameters. 

Let's use the Sampler to sample the quasi distribution for every parameter theta.

In [ ]:
# Define a range of parameters (phases) between 0 and 2 pi

# Your code goes here

In [ ]:
qc.measure_all()
qc.draw()

**Note:**

Using a Runtime Session we can evaluate the circuit for all parameters in one go, i.e. we only need to queue once!

In [ ]:
# Before we run the circuit, we have to transpile it to the instruction set architecture of the backend of choice.
pm = generate_preset_pass_manager(backend=ibmq_backend, optimization_level=1)
isa_qc = pm.run(qc)
isa_qc.draw(idle_wires=False)

In [ ]:
# Use the Session context manager to execute the quntum circuit on a real device
# via the Sampler runtime primitive. 

# Your code goes here

In [ ]:
# Retrieve job results
result = service.job('cqxdcjtp3cg0008ch7f0').result()

In [ ]:
import matplotlib.pyplot as plt

# The probablity of measuring |0> for each theta
prob_values_0 = [dist.get(0, 0) for dist in result.quasi_dists]
# The probablity of measuring |1> for each theta
prob_values_1 = [dist.get(1, 0) for dist in result.quasi_dists]

plt.plot(phases, prob_values_0, 'o', label=r'$P_0 = |<0|\psi(\theta)>|^2 = |<0|R_y(\theta)|0>|^2 = \cos^2\frac{\theta}{2}$')
plt.plot(phases, prob_values_1, 'o', label=r'$P_1 = |<1|\psi(\theta)>|^2 = |<1|R_y(\theta)|0>|^2 = \sin^2\frac{\theta}{2}$')
plt.xlabel(r'Phase $\theta$')
plt.ylabel(r'Probability')
plt.legend()

## Dynamic Circuits

**Dynamic circuits** are quantum circuits that contain mid-circuit measurements where the results of those measurements are used to condition quantum gates later in the circuit. The ability to condition future quantum operations on the classical measurement results is known as classical feedforward.

Dynamic circuits are quantum circuits that include control flow such as if statements and while loops

In [ ]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr, cr)

q0, q1 = qr
b0, b1 = cr

qc.h(q0)
qc.measure(q0, b0)

with qc.if_test((b0, 0)) as else_:
    qc.x(q1)
with else_:
    qc.h(q1)

qc.measure(q1, b1)
qc.draw(idle_wires=False)

**Question:**

Considering the circuit above:

What are (roughly) the expected counts for the $|00\rangle$, $|01\rangle$, $|10\rangle$, $|11\rangle$ states?

In [ ]:
# Execute the dynamic circuit (qc) on a local backend. Extract the counts.

# Your code goes here


## Quantum Teleportation

Alice possesses a qubit in an unknown state $\lvert \psi \rangle$ and she wishes to transfer this quantum state to Bob. She can not simply clone or copy the state, but she can transfer the her qubit state to Bob.

By sending two bits of classical information Bob will at the end possess $\lvert \psi \rangle$, and Alice will no longer have it. This is known as teleportation.

The protocol requires three qubits:

1. The qubit to be teleported (Alice's qubit)
2. One half of an entangled pair of qubits (Alice's second qubit)
3. The other half of the entangled pair (Bob's qubit)

The protocol can be summarized in the following steps:

1. Create an entangled pair of qubits (Bell pair) shared between Alice and Bob.
2. Alice performs a Bell basis measurement on her two qubits.
3. Alice sends the classical results of her measurement to Bob.
4. Bob applies appropriate quantum gates based on Alice's measurement results to obtain the teleported state.

In [ ]:
Image(filename='images/quantum_teleportation.jpeg') 

In [ ]:
qr = QuantumRegister(3, name="q")
cr = ClassicalRegister(3, name="c")
s, a, b = qr
c0, c1, c2 = cr

In [ ]:
def create_bell_pair(qr: QuantumRegister, cr: ClassicalRegister) -> QuantumCircuit:
    """Creates a bell pair between qubits a and b."""
    qc = QuantumCircuit(qr, cr)
    # the first qubit is s but we won't be using it in this exercise
    s, a, b = qr
    # Create a bell pair between alice and bob.

    # Your code goes here
    
    return qc

In [ ]:
def alice_gates(qr: QuantumRegister, cr: ClassicalRegister) -> QuantumCircuit:
    """Creates Alices's gates"""
    qc = QuantumCircuit(qr, cr)
    s, a, b = qr
    # Perform a bell basis measurement on alices qubits (s,a)

    # Your code goes here
    
    return qc

In [ ]:
def measure_and_send(qr: QuantumRegister, cr: ClassicalRegister):
    """Measures qubits a & b and 'sends' the results to Bob"""
    qc = QuantumCircuit(qr, cr)
    s, a, b = qr
    c0, c1, c2 = cr
    qc.measure([a,s],[c0,c1])
    return qc

In [ ]:
def bob_gates(qr: QuantumRegister, cr: ClassicalRegister):
    """Uses qc.if_test to control which gates are dynamically added"""
    qc = QuantumCircuit(qr, cr)
    s, a, b = qr
    c0, c1, c2 = cr
    # If the bits are `00`, no action is required.
    # If they are `01`, an 𝑋 gate (also known as a Pauli-X or a bit-flip gate) should be applied.
    # For bits `10`, a 𝑍 gate (also known as a Pauli-Z or a phase-flip gate) should be applied. 
    # Lastly, if the classical bits are `11`, a combined 𝑍𝑋 sequence should be applied.

    # Your code goes here
    
    return qc

In [ ]:
teleport = create_bell_pair(qr,cr)
teleport.compose(alice_gates(qr,cr),inplace=True)
teleport.compose(measure_and_send(qr,cr),inplace=True)
teleport.compose(bob_gates(qr,cr),inplace=True)
teleport.measure(b,c2)
teleport.draw()

In [ ]:
# define source qubit
source = QuantumCircuit(qr,cr)
source.ry(np.pi/4,0)
source.draw()

In [ ]:
teleport_source = source.compose(teleport)
teleport_source.draw()

In [ ]:
# run job source 
source.measure_all()
counts_source = aer_backend.run(source, shots=4000).result().get_counts()

In [ ]:
plot_histogram(counts_source)

In [ ]:
# Transpile the circuit.
isa_teleport_source = pm.run(teleport_source)

In [ ]:
# run teleport source
job_teleport_source = ibmq_backend.run(isa_teleport_source, dynamic=True)
job_teleport_source

In [ ]:
counts_teleport_source = service.job('cqxdx34hepxg008182p0').result().get_counts()

In [ ]:
# Get the results and display them
plot_histogram(counts_teleport_source)

In [ ]:
bobs_counts = marginal_counts(counts_teleport_source, [qr.index(b)])
plot_histogram(bobs_counts)